In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import sklearn.cross_validation
from sklearn.cross_validation import cross_val_score

In [416]:
def setPred(col):
    if(col > 0):
        return 1
    else:
        return 0

# preparar/tweks datos 

In [417]:
features = pd.read_csv("features0,0.csv")
prediccion = pd.read_csv("csv/labels_training_set.csv")

In [418]:
#mergeo features y la info q nos tiran, lleno con 0 los q no se
features = pd.merge(prediccion,features,on="person",how= "left").dropna()

In [419]:
prediccion = features["label"]

In [420]:
features = features.drop(["person","Unnamed: 0","label"],axis = 1)

In [421]:
features = features.drop(["level_0"],axis = 1)

In [422]:
for word in features.columns:
    features[word] = pd.to_numeric(features[word],errors = "coerce")

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(features, prediccion, test_size=0.30, random_state=42)

# Random Forest
# historial cambios tweaks | score

columnas: 
'ad campaign hit', 'brand listing', 'checkout', 'generic listing',
       'lead', 'search engine hit', 'searched products', 'staticpage',
       'viewed product', 'visited site', 'level_0', 'storage_no_convercion',
       'numero mas usado dia', 'condition_no_convercion2',
       'color_no_convercion2', 'search_engine2', 'nombre mas usado dia2']

test_size = 0.3
cv = 20
n_estimator 50 | .78 (Kaggle .76)
n_estimator 100 | .79 


https://www.youtube.com/watch?v=BSUMBBFjxrY

In [424]:
randomforesttree = RandomForestRegressor(random_state = None, n_jobs=-1)
randomforesttree.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [462]:
param_grid= {'max_features': ["auto"], 'n_estimators': [100], 'criterion': ['mse']}
grid_drop = GridSearchCV(randomforesttree, param_grid, cv=20, scoring='roc_auc',n_jobs = -1,pre_dispatch = 4)
grid_drop.fit(X_train, y_train)
print(grid_drop.best_params_, grid_drop.best_score_)

JoblibException: JoblibException
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.5/site-packages/ipykernel_launcher.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.5/runpy.py in _run_code(code=<co

In [ ]:
result = grid_drop.best_estimator_.predict(X_test)

In [ ]:
roc_auc_score(y_test, result)

In [431]:
feature_importances = pd.DataFrame(grid_drop.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [432]:
feature_importances

,importance
condition_no_convercion2,0.124678
color_no_convercion2,0.115660
nombre mas usado dia2,0.110516
search_engine2,0.104809
checkout,0.082547
viewed product,0.070564
numero mas usado dia,0.067453
brand listing,0.055407
visited site,0.048127
generic listing,0.044892


# Gradient Boost
# historial cambios tweaks | score 



In [283]:
gradientboost = GradientBoostingClassifier(random_state = None)
gradientboost.get_params()

{'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'presort': 'auto',
 'random_state': None,
 'subsample': 1.0,
 'verbose': 0,
 'warm_start': False}

In [284]:
param_grid= {'max_features': ["auto"], 'n_estimators': [30], 'loss': ['deviance', 'exponential']}
grid_drop = GridSearchCV(gradientboost, param_grid, cv=20, scoring='roc_auc')
grid_drop.fit(X_train, y_train)
print(grid_drop.best_params_, grid_drop.best_score_)

{'max_features': 'auto', 'loss': 'exponential', 'n_estimators': 30} 0.8274479183851948


In [285]:
result2 = grid_drop.best_estimator_.predict(X_test)

In [286]:
roc_auc_score(y_test, result2)

0.5

In [287]:
feature_importances2 = pd.DataFrame(grid_drop.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
checkout,0.138690
color_no_convercion2,0.095729
search_engine2,0.095588
condition_no_convercion2,0.092753
nombre mas usado dia2,0.088604
viewed product,0.079509
numero mas usado dia,0.073116
visited site,0.052279
brand listing,0.051483
generic listing,0.051276


# Summit



In [445]:
to_pred = pd.read_csv("csv/trocafone_kaggle_test.csv")
features = pd.read_csv("features0,0.csv")

In [446]:
features = features.drop(["Unnamed: 0"],axis = 1)

In [447]:
features = features.drop(["level_0"],axis = 1)


In [448]:
to_pred_completo = pd.merge(to_pred,features,on="person",how = "left").dropna()

In [449]:
to_pred_completo["label"] = grid_drop.best_estimator_.predict(to_pred_completo.drop(["person"],axis = 1))

In [450]:
to_pred = pd.merge(to_pred_completo[["person","label"]],to_pred,on="person",how="right")

In [454]:
to_pred = to_pred.fillna(to_pred["label"].median())

In [458]:
to_pred.to_csv("sumit0.0", index=False)

5